In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [2]:
df=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
df.head()

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911


In [39]:
test.head()

,row_id,date,country,store,product
0,26298,2019-01-01,Finland,KaggleMart,Kaggle Mug
1,26299,2019-01-01,Finland,KaggleMart,Kaggle Hat
2,26300,2019-01-01,Finland,KaggleMart,Kaggle Sticker
3,26301,2019-01-01,Finland,KaggleRama,Kaggle Mug
4,26302,2019-01-01,Finland,KaggleRama,Kaggle Hat


In [7]:
df.shape

(26298, 6)

In [6]:
df.dtypes

row_id       int64
date        object
country     object
store       object
product     object
num_sold     int64
dtype: object

I'll try lstm afterwards because there's time series. 

In [8]:
df1=df.drop(['row_id','date'],axis=1)
test11=test.drop(['row_id','date'],axis=1)

In [9]:
df1.head()

,country,store,product,num_sold
0,Finland,KaggleMart,Kaggle Mug,329
1,Finland,KaggleMart,Kaggle Hat,520
2,Finland,KaggleMart,Kaggle Sticker,146
3,Finland,KaggleRama,Kaggle Mug,572
4,Finland,KaggleRama,Kaggle Hat,911


In [16]:
df2.head()

,country,store,product,num_sold
0,0,0,1,329
1,0,0,0,520
2,0,0,2,146
3,0,1,1,572
4,0,1,0,911


In [13]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df2=df1.copy()
df2['country']=le.fit_transform(df1['country'])
df2['store']=le.fit_transform(df1['store'])
df2['product']=le.fit_transform(df1['product'])

test2=test11.copy()
test2['country']=le.fit_transform(test11['country'])
test2['store']=le.fit_transform(test11['store'])
test2['product']=le.fit_transform(test11['product'])

In [17]:
X=df2.drop(['num_sold'],axis=1)
y=df2['num_sold']

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR 
from sklearn.model_selection import cross_val_score

In [20]:
a=[LinearRegression(),RandomForestRegressor(),AdaBoostRegressor(),ExtraTreesRegressor(),DecisionTreeRegressor(),SVR()] #runnning other regressors takes up too much computational power ig
avglist=[]
templist=[]
avg=0
for i in range(len(a)):
    temp=cross_val_score(a[i],X,y)
    templist.append(temp)
    for i in temp:
        avg+=i
    avg=avg/5
    avglist.append(avg)
    avg=0
res = dict(zip(a, avglist))
res

{LinearRegression(): 0.6344403180943162,
 RandomForestRegressor(): 0.8023871842705705,
 AdaBoostRegressor(): 0.6677211572510661,
 ExtraTreesRegressor(): 0.8024100900898683,
 DecisionTreeRegressor(): 0.8024100900898684,
 SVR(): 0.7766346797686012}

In [21]:
X.shape

(26298, 3)

In [38]:
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(loss='least_squares', learning_rate=0.3, max_iter=1000, max_leaf_nodes=30,).fit(X, y)
print(model.score(X, y))


c:\Python310\lib\site-packages\sklearn\ensemble\_hist_gradient_boosting\gradient_boosting.py:1412: FutureWarning: The loss 'least_squares' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(


0.8047604968244065


In [40]:
Y_proba = model.predict(test2)
submission = pd.DataFrame(data={'row_id': test.row_id, 'num_sold': Y_proba})
submission.head()


,row_id,num_sold
0,26298,203.797102
1,26299,362.494331
2,26300,103.175479
3,26301,356.201533
4,26302,629.862835


In [41]:
submission.to_csv('submission.csv', index=False)

In [22]:
import tensorflow
import tensorflow.keras as keras

In [34]:
mod2=keras.Sequential([
    keras.layers.Dense(3,input_shape=(3,),activation='relu'),
    keras.layers.Dense(3,activation='relu'),     
    keras.layers.Dense(1)])
mod2.compile(optimizer='adam',metrics=['accuracy'],loss='MeanSquaredError')


In [35]:
mod2.fit(X,y,epochs=5)

Epoch 1/5
822/822 [==============================] - 6s 7ms/step - loss: 214110.0156 - accuracy: 0.0000e+00
Epoch 2/5
822/822 [==============================] - 5s 6ms/step - loss: 161941.1406 - accuracy: 0.0000e+00
Epoch 3/5
822/822 [==============================] - 5s 6ms/step - loss: 107100.8672 - accuracy: 0.0000e+00
Epoch 4/5
822/822 [==============================] - 5s 6ms/step - loss: 83039.1953 - accuracy: 0.0000e+00
Epoch 5/5
822/822 [==============================] - 5s 6ms/step - loss: 68311.4062 - accuracy: 0.0000e+00
